In [1]:
pip install transformers[torch]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 3.3 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-m

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [25]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from transformers import GPT2Tokenizer, GPT2ForSequenceClassification, Trainer, TrainingArguments
import torch
from torch.utils.data import Dataset


#Data processing


***Reading the dataset***

In [26]:
#Reading the csv
df = pd.read_csv('/content/drive/MyDrive/TrabajoFinal/cyberbullying_tweets.csv', names=['text', 'cyberbullying_type'])


total_nulls = df.isnull().sum().sum()

print("Null data:", total_nulls)

# Making sure is a string (just in case)
df['cyberbullying_type'] = df['cyberbullying_type'].astype(str)

# Select 200 text for each label
df_sampled = df.groupby('cyberbullying_type').apply(lambda x: x.sample(n=min(len(x), 300), random_state=42)).reset_index(drop=True)

# Drop the categories we don't plan to use
df_sampled = df_sampled[~df_sampled['cyberbullying_type'].isin(['cyberbullying_type', 'other_cyberbullying'])]

Null data: 0


***Getting the labels***

In [27]:
# Convert to categorical and set a index number for the selected labels
df_sampled['cyberbullying_type'] = df_sampled['cyberbullying_type'].astype('category')
df_sampled['label'] = df_sampled['cyberbullying_type'].cat.codes

# Crear el mapeo de categorías a códigos numéricos
category_mapping = dict(enumerate(df_sampled['cyberbullying_type'].cat.categories))

print("Ciberbullying numerical labels:")
for code, category in category_mapping.items():
    print(f"{code}: {category}")


Ciberbullying numerical labels:
0: age
1: ethnicity
2: gender
3: not_cyberbullying
4: religion


***Getting the train and test values***

In [28]:
# Split data
train_df, test_df = train_test_split(df_sampled, test_size=0.2, random_state=42)

***Tokenization using GPT-2***

In [29]:
# GPT-2 Tokenizator
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
tokenizer.add_special_tokens({'pad_token': '[PAD]'})
tokenizer.pad_token = '[PAD]'

***Preparing the data in order to train and validate***

In [30]:
class CyberbullyingDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length=128):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]
        encoding = self.tokenizer(
            text,
            truncation=True,
            padding='max_length',
            max_length=self.max_length,
            return_tensors='pt'
        )
        input_ids = encoding['input_ids'].flatten()
        attention_mask = encoding['attention_mask'].flatten()
        return {
            'input_ids': input_ids,
            'attention_mask': attention_mask,
            'labels': torch.tensor(label, dtype=torch.long)
        }


***Calling the class we made***

In [31]:
# Creating the datasets for the training and tests
train_dataset = CyberbullyingDataset(
    texts=train_df['text'].tolist(),
    labels=train_df['label'].tolist(),
    tokenizer=tokenizer
)

test_dataset = CyberbullyingDataset(
    texts=test_df['text'].tolist(),
    labels=test_df['label'].tolist(),
    tokenizer=tokenizer
)

***Instantiating the model***

In [32]:
# Definir el modelo GPT-2 para clasificación de secuencias y ajustar los tokens especiales
model = GPT2ForSequenceClassification.from_pretrained('gpt2', num_labels=df_sampled['cyberbullying_type'].nunique())
model.resize_token_embeddings(len(tokenizer))

# Establecer el token de padding en la configuración del modelo
model.config.pad_token_id = tokenizer.pad_token_id

# Mover el modelo al mismo dispositivo
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


GPT2ForSequenceClassification(
  (transformer): GPT2Model(
    (wte): Embedding(50258, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (score): Linear(in_features=768, out_features=5, bias=False)
)

***Defining and training the model***

In [33]:
# Definir los argumentos de entrenamiento
training_args = TrainingArguments(
    output_dir='./results',          # Directorio de salida para los checkpoints y los logs
    num_train_epochs=5,              # Número total de epochs de entrenamiento
    per_device_train_batch_size=8,   # Tamaño del batch de entrenamiento por dispositivo
    per_device_eval_batch_size=8,    # Tamaño del batch de evaluación por dispositivo
    warmup_steps=500,                # Número de pasos para el calentamiento del optimizador
    weight_decay=0.01,               # Factor de decaimiento para el ajuste de los pesos
    logging_dir='./logs',            # Directorio de logs
    evaluation_strategy='epoch'      # Evaluar el modelo al final de cada epoch
)


# Crear el objeto Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset
)

# Entrenar el modelo
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,No log,1.320223
2,No log,0.398461
3,No log,0.388760
4,1.099100,0.341385
5,1.099100,0.435851


TrainOutput(global_step=750, training_loss=0.7674825337727864, metrics={'train_runtime': 192.7859, 'train_samples_per_second': 31.123, 'train_steps_per_second': 3.89, 'total_flos': 391955742720000.0, 'train_loss': 0.7674825337727864, 'epoch': 5.0})

***Evaluation and proccess to save the trained model***

In [34]:
# Evaluar el modelo en el conjunto de prueba
predictions = trainer.predict(test_dataset)
predicted_classes = np.argmax(predictions.predictions, axis=1)
true_classes = test_df['label'].values

# Mostrar métricas de clasificación
print(classification_report(true_classes, predicted_classes))

# Guardar el modelo entrenado
model.save_pretrained('./gpt2_cyberbullying_model')
tokenizer.save_pretrained('./gpt2_cyberbullying_tokenizer')

              precision    recall  f1-score   support

           0       0.95      0.95      0.95        56
           1       0.98      0.94      0.96        65
           2       0.80      0.92      0.86        49
           3       0.86      0.77      0.82        66
           4       0.91      0.95      0.93        64

    accuracy                           0.90       300
   macro avg       0.90      0.91      0.90       300
weighted avg       0.91      0.90      0.90       300



('./gpt2_cyberbullying_tokenizer/tokenizer_config.json',
 './gpt2_cyberbullying_tokenizer/special_tokens_map.json',
 './gpt2_cyberbullying_tokenizer/vocab.json',
 './gpt2_cyberbullying_tokenizer/merges.txt',
 './gpt2_cyberbullying_tokenizer/added_tokens.json')

***Using the model in inference***

In [42]:
# Hacer inferencias con el modelo entrenado
text_to_predict = "I was bullied for not having a gf throughout my high school years. Even got called gay and what not. Joke was on them when I showed up to my senior prom with a college girl"
encoded_text = tokenizer(text_to_predict, truncation=True, padding=True, return_tensors="pt")

# Mover los tensores a la misma GPU que el modelo si es necesario
encoded_text = {k: v.to(device) for k, v in encoded_text.items()}

logits = model(**encoded_text).logits
predicted_class = torch.argmax(logits, dim=1).item()

# Mostrar la clase predicha junto con su descripción
predicted_category = category_mapping[predicted_class]
print("Clase predicha:", predicted_class, "-", predicted_category)

Clase predicha: 0 - age
